### Summary
* GA4 sample 데이터 스키마를 기반. 해당 테이블을 기준으로 프롬프트 질의 시 질의에 정답에 해당하는 sql query 문 생성

### Process
1. 문서의 내용을 읽는다.
2. 문서를 쪼갠다 (chunking)
3. 문서를 임베딩 해서 벡터 데이터베이스 저장 (Pinecone)
4. 질문이 있을 때 벡터 데이터베이스에서 유사도를 검색
5. 유사도 검색으로 가져온 문서를 LLM에 질문과 같이 전달

In [2]:
import getpass
import os

from langchain_community.document_loaders import Docx2txtLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from dotenv import load_dotenv
from langchain_openai import OpenAIEmbeddings

from pinecone import Pinecone
from langchain_pinecone import PineconeVectorStore
from langchain_openai import ChatOpenAI
from langchain.chains import RetrievalQA
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate

load_dotenv()
embedding = OpenAIEmbeddings(model = 'text-embedding-3-large')
llm = ChatOpenAI(model ='gpt-4o')

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 1000, # chunk 하나가 가질 수 있는 token 수
    # chunk_overlap = 50 # chunk 간 token 을 겹치게 하는 범위 (유사도 검색의 성능을 더 올림)
)

loader = Docx2txtLoader('./GA4_schema_markdown.docx')
document_list = loader.load_and_split(text_splitter=text_splitter)

/Users/a202304035/.pyenv/versions/3.11.0/envs/llm_prototype/lib/python3.11/site-packages/pinecone/data/index.py:1: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [3]:
len(document_list)

39

In [4]:
print(document_list[0].page_content)

[GA4] BigQuery Export schema

This article explains the format and schema of the Google Analytics 4 property data and the Google Analytics for Firebase data that is exported to BigQuery.

Datasets

For each Google Analytics 4 property and each Firebase project that is linked to BigQuery, a single dataset named "analytics_<property_id>" is added to your BigQuery project. Property ID refers to your Analytics Property ID, which you can find in the property settings for your Google Analytics 4 property, and in App Analytics Settings in Firebase. Each Google Analytics 4 property and each app for which BigQuery exporting is enabled will export its data to that single dataset.

Tables

Within each dataset, a table named events_YYYYMMDD is created each day if the Daily export option is enabled.


In [5]:
if not os.getenv("PINECONE_API_KEY"):
    os.environ["PINECONE_API_KEY"] = getpass.getpass("Enter your Pinecone API key: ")

index_name = 'ga4-sql-index'
pinecone_api_key = os.environ.get("PINECONE_API_KEY")

pc = Pinecone(api_key=pinecone_api_key)

database = PineconeVectorStore.from_documents(document_list, embedding, index_name=index_name)

In [6]:
# query = '2024년 11월 1일 기준으로, device tpye 별로 unique user 의 수를 집계하는 쿼리를 작성해주세요.'
query = 'event 가 발생한 날짜를 2020년 12월 한 달로 설정하고, device tpye 별로 전체 구분 가능한 user 의 수와 구매자의 수, 해당 기간 동안의 총 구매액 합계를 추출하는 쿼리를 작성해주세요.'
retrieved_docs = database.similarity_search(query, k= 7)

In [7]:
retrieved_docs

[Document(id='6b3a39e4-9f2a-4afa-af57-61c6b6f82dc0', metadata={'source': './GA4_schema_markdown.docx'}, page_content='| Field name | Data type | Description |\n| ecommerce.total_item_quantity | INTEGER | Total number of items in this event, which is the sum of items.quantity. |\n| ecommerce.purchase_revenue_in_usd | FLOAT | Purchase revenue of this event, represented in USD with standard unit. Populated for purchase event only. |\n| ecommerce.purchase_revenue | FLOAT | Purchase revenue of this event, represented in local currency with standard unit. Populated for purchase event only. |'),
 Document(id='dc8e1ea3-9816-4fbd-b8ee-fea52924367a', metadata={'source': './GA4_schema_markdown.docx'}, page_content='Tables\n\nWithin each dataset, a table named\xa0events_YYYYMMDD\xa0is created each day if the Daily export option is enabled.\n\nIf the Streaming export option is enabled, a table named\xa0events_intraday_YYYYMMDD\xa0is created. This table is populated continuously as events are record

In [8]:
prompt = f"""[identity]
- 당신은 데이터 과학자이자 데이터 분석가입니다. 주어진 데이터에 대한 정보를 바탕으로 데이터를 활용하고자 하는 사람들의 질문에 답할 수 있는 쿼리를 작성할 수 있습니다. 
- 특히 Bigquery 환경에서 필요한 테이블을 추출하는데 어려움이 없습니다.
- [Context]를 참고해서 사용자의 질문에 답변해주세요.

[Context]
{retrieved_docs}

Question: {query}
"""

In [9]:
ai_message = llm.invoke(prompt)

In [10]:
print(ai_message.content)

To address your request, we will create a BigQuery SQL query that extracts data for events happening in December 2020, grouped by device type. We will calculate the distinct number of users, the number of purchasers, and the total purchase revenue for that period. Here's how you can structure the query:

```sql
SELECT
  device.type AS device_type,
  COUNT(DISTINCT user_pseudo_id) AS total_users,
  COUNT(DISTINCT CASE WHEN ecommerce.purchase_revenue IS NOT NULL THEN user_pseudo_id END) AS total_purchasers,
  SUM(ecommerce.purchase_revenue_in_usd) AS total_purchase_revenue
FROM
  `your_project.your_dataset.events_*`
WHERE
  _TABLE_SUFFIX BETWEEN '20201201' AND '20201231' -- Filter for December 2020
  AND event_name = 'purchase' -- Considering only purchase events for revenue
GROUP BY
  device_type
ORDER BY
  total_purchase_revenue DESC;
```

### Explanation:
- **`device.type`**: Assumed field for device type. You might need to adjust based on your schema.
- **`user_pseudo_id`**: A unique